# 1. Download/Load SP500 stocks prices data.

In [ ]:
!pip install statsmodels
!pip install matplotlib
!pip install pandas
!pip install numpy
!pip install pandas-datareader
!pip install yfinance
!pip install datetime
!pip install scikit-learn
!pip install PyPortfolioOpt
!pip install pandas-ta

In [ ]:
# Reinstalar pandas-ta con una versión específica compatible con numpy
!pip uninstall -y pandas-ta
!pip install pandas-ta==0.3.14b0

In [4]:
# Corregir el error de importación en pandas-ta
import os
import site

# Buscar el archivo directamente en site-packages
site_packages = site.getsitepackages()[0]
squeeze_pro_path = os.path.join(site_packages, 'pandas_ta', 'momentum', 'squeeze_pro.py')

# Verificar si el archivo existe
if os.path.exists(squeeze_pro_path):
    print(f"Archivo encontrado en: {squeeze_pro_path}")
    
    # Leer el contenido actual
    with open(squeeze_pro_path, 'r') as file:
        content = file.read()
    
    # Reemplazar 'from numpy import NaN as npNaN' por 'from numpy import nan as npNaN'
    if 'from numpy import NaN as npNaN' in content:
        content = content.replace('from numpy import NaN as npNaN', 'from numpy import nan as npNaN')
        
        # Guardar el archivo modificado
        with open(squeeze_pro_path, 'w') as file:
            file.write(content)
        print("El archivo ha sido corregido correctamente.")
    else:
        print("El archivo ya ha sido corregido o tiene un formato diferente.")
else:
    print(f"Buscando archivo en: {site_packages}")
    
    # Buscar el archivo en todo el site-packages
    found = False
    for root, dirs, files in os.walk(site_packages):
        if 'squeeze_pro.py' in files:
            found_path = os.path.join(root, 'squeeze_pro.py')
            print(f"Archivo encontrado en ubicación alternativa: {found_path}")
            
            # Leer el contenido
            with open(found_path, 'r') as file:
                content = file.read()
            
            if 'from numpy import NaN as npNaN' in content:
                content = content.replace('from numpy import NaN as npNaN', 'from numpy import nan as npNaN')
                
                # Guardar el archivo modificado
                with open(found_path, 'w') as file:
                    file.write(content)
                print("El archivo ha sido corregido correctamente.")
                found = True
                break
            else:
                print("El archivo ya ha sido corregido o tiene un formato diferente.")
                found = True
                break
    
    if not found:
        print("No se pudo encontrar el archivo 'squeeze_pro.py' en ninguna ubicación.")


Buscando archivo en: c:\Users\ASUS\Desktop\Juanes\Infraestructura\Proyecto-test1\.venv
Archivo encontrado en ubicación alternativa: c:\Users\ASUS\Desktop\Juanes\Infraestructura\Proyecto-test1\.venv\Lib\site-packages\pandas_ta\momentum\squeeze_pro.py
El archivo ha sido corregido correctamente.


In [13]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

print(f"Número de símbolos: {len(symbols_list)}")
print(f"Primeros 10 símbolos: {symbols_list[:10]}")

end_date = '2025-05-14'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

print(f"\nDescargando datos desde {start_date} hasta {end_date}")

# Descargar los datos con parámetros específicos para preservar todas las columnas
df_raw = yf.download(tickers=symbols_list,
                     start=start_date,
                     end=end_date,
                     auto_adjust=False,  # Importante: no ajustar automáticamente
                     prepost=True,
                     threads=True)

print(f"\nForma de df_raw: {df_raw.shape}")
print(f"Columnas de df_raw: {df_raw.columns.tolist()}")

# Verificar si tenemos datos
if df_raw.empty:
    print("¡ERROR: No se pudieron descargar datos!")
else:
    # Reorganizar el DataFrame correctamente
    df = df_raw.stack(level=1)  # Stack el nivel de los tickers
    df.index.names = ['date', 'ticker']
    
    # Convertir nombres de columnas a minúsculas
    df.columns = df.columns.str.lower()
    
    print(f"\nForma de df después de stack: {df.shape}")
    print(f"Columnas de df: {df.columns.tolist()}")
    print(f"Nombres del índice: {df.index.names}")
    
    # Verificar que tenemos adj close
    if 'adj close' in df.columns:
        print("\n✓ Columna 'adj close' encontrada correctamente")
        print(f"Primeros valores de 'adj close':")
        print(df['adj close'].dropna().head())
    else:
        print("\n✗ Columna 'adj close' NO encontrada")
        print("Columnas disponibles:", df.columns.tolist())

df


Número de símbolos: 503
Primeros 10 símbolos: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

Descargando datos desde 2017-05-16 00:00:00 hasta 2025-05-14


[*********************100%***********************]  503 of 503 completed




Forma de df_raw: (2010, 3018)
Columnas de df_raw: [('Adj Close', 'A'), ('Adj Close', 'AAPL'), ('Adj Close', 'ABBV'), ('Adj Close', 'ABNB'), ('Adj Close', 'ABT'), ('Adj Close', 'ACGL'), ('Adj Close', 'ACN'), ('Adj Close', 'ADBE'), ('Adj Close', 'ADI'), ('Adj Close', 'ADM'), ('Adj Close', 'ADP'), ('Adj Close', 'ADSK'), ('Adj Close', 'AEE'), ('Adj Close', 'AEP'), ('Adj Close', 'AES'), ('Adj Close', 'AFL'), ('Adj Close', 'AIG'), ('Adj Close', 'AIZ'), ('Adj Close', 'AJG'), ('Adj Close', 'AKAM'), ('Adj Close', 'ALB'), ('Adj Close', 'ALGN'), ('Adj Close', 'ALL'), ('Adj Close', 'ALLE'), ('Adj Close', 'AMAT'), ('Adj Close', 'AMCR'), ('Adj Close', 'AMD'), ('Adj Close', 'AME'), ('Adj Close', 'AMGN'), ('Adj Close', 'AMP'), ('Adj Close', 'AMT'), ('Adj Close', 'AMZN'), ('Adj Close', 'ANET'), ('Adj Close', 'ANSS'), ('Adj Close', 'AON'), ('Adj Close', 'AOS'), ('Adj Close', 'APA'), ('Adj Close', 'APD'), ('Adj Close', 'APH'), ('Adj Close', 'APO'), ('Adj Close', 'APTV'), ('Adj Close', 'ARE'), ('Adj Clos

Price               adj close       close        high         low        open  \
date       ticker                                                               
2017-05-16 A        53.492157   56.830002   58.380001   56.450001   58.380001   
           AAPL     36.213570   38.867500   39.014999   38.680000   38.985001   
           ABBV     47.478668   66.849998   66.919998   66.330002   66.730003   
           ABT      37.904308   43.689999   44.590000   43.410000   44.590000   
           ACGL     30.206877   31.766666   31.863333   31.590000   31.790001   
...                       ...         ...         ...         ...         ...   
2025-05-13 XYL     126.930000  126.930000  127.709999  126.779999  127.180000   
           YUM     145.720001  145.720001  147.009995  144.809998  147.000000   
           ZBH      97.489998   97.489998   98.230003   96.360001   97.989998   
           ZBRA    302.380005  302.380005  305.290009  300.369995  300.750000   
           ZTS     160.410004  160.410004  162.910004  160.059998  160.720001   

Price                  volume  
date       ticker              
2017-05-16 A        1032600.0  
           AAPL    80194000.0  
           ABBV     6313500.0  
           ABT      7159300.0  
           ACGL      804600.0  
...                       ...  
2025-05-13 XYL      1327600.0  
           YUM      1880000.0  
           ZBH      2324000.0  
           ZBRA      846800.0  
           ZTS      3336700.0  

[990380 rows x 6 columns]

# 2. Calculate features and technical indicators for each stock.
- Garman-Klass Volatility
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

 \begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}
    

In [14]:
# Calcular Garman-Klass Volatility
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# Calcular RSI
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# Calcular Bollinger Bands
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

# Calcular ATR normalizado
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['adj close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# Calcular MACD normalizado
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# Calcular volumen en millones de dólares
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

# Mostrar el DataFrame con los indicadores calculados
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2017-05-16 A        53.492157   56.830002   58.380001   56.450001   58.380001   
           AAPL     36.213570   38.867500   39.014999   38.680000   38.985001   
           ABBV     47.478668   66.849998   66.919998   66.330002   66.730003   
           ABT      37.904308   43.689999   44.590000   43.410000   44.590000   
           ACGL     30.206877   31.766666   31.863333   31.590000   31.790001   
...                       ...         ...         ...         ...         ...   
2025-05-13 XYL     126.930000  126.930000  127.709999  126.779999  127.180000   
           YUM     145.720001  145.720001  147.009995  144.809998  147.000000   
           ZBH      97.489998   97.489998   98.230003   96.360001   97.989998   
           ZBRA    302.380005  302.380005  305.290009  300.369995  300.750000   
           ZTS     160.410004  160.410004  162.910004  160.059998  160.720001   

Price                  volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2017-05-16 A        1032600.0         -0.002388        NaN       NaN   
           AAPL    80194000.0         -0.002064        NaN       NaN   
           ABBV     6313500.0         -0.044715        NaN       NaN   
           ABT      7159300.0         -0.009834        NaN       NaN   
           ACGL      804600.0         -0.000971        NaN       NaN   
...                       ...               ...        ...       ...   
2025-05-13 XYL      1327600.0          0.000025  61.794912  4.678320   
           YUM      1880000.0          0.000084  45.965021  4.962103   
           ZBH      2324000.0          0.000175  44.094881  4.524274   
           ZBRA      846800.0          0.000121  63.240049  5.357481   
           ZTS      3336700.0          0.000154  53.479526  4.982374   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2017-05-16 A            NaN       NaN       NaN       NaN      55.236001  
           AAPL         NaN       NaN       NaN       NaN    2904.111004  
           ABBV         NaN       NaN       NaN       NaN     299.756572  
           ABT          NaN       NaN       NaN       NaN     271.368315  
           ACGL         NaN       NaN       NaN       NaN      24.304453  
...                     ...       ...       ...       ...            ...  
2025-05-13 XYL     4.777212  4.876105 -1.323269  1.367672     168.512268  
           YUM     4.994632  5.027161 -1.682124 -0.602270     273.953602  
           ZBH     4.595950  4.667626 -1.120584 -0.988090     226.566755  
           ZBRA    5.522473  5.687466  0.929265  0.755502     256.055388  
           ZTS     5.043721  5.105067 -0.784831  0.327079     535.240059  

[990380 rows x 14 columns]

# 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.
- To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [15]:
last_cols = [c for c in df.columns.unique(0) if c not in ['open', 'high', 'low', 'close', 'volume','dollar_volume']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                    df.unstack()[last_cols].resample('M').last().stack('ticker')],
                   axis=1)).dropna()

data
#df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2017-06-30 A          109.548804   55.950832         -0.001012  53.174485   
           AAPL      4268.541570   33.546528         -0.002065  38.245982   
           ABBV       298.847267   51.498539         -0.045411  73.663831   
           ABT        302.877918   42.172794         -0.008142  70.962073   
           ACGL        69.558512   29.569775         -0.001103  42.661682   
...                          ...         ...               ...        ...   
2025-05-31 XYL        175.917955  126.930000          0.000025  61.794912   
           YUM        269.926197  145.720001          0.000084  45.965021   
           ZBH        321.604173   97.489998          0.000175  44.094881   
           ZBRA       206.729977  302.380005          0.000121  63.240049   
           ZTS        590.935495  160.410004          0.000154  53.479526   

                     bb_low    bb_mid   bb_high       atr      macd  
date       ticker                                                    
2017-06-30 A       4.027342  4.046235  4.065128 -0.308418  0.007961  
           AAPL    3.511176  3.563069  3.614962 -1.023575 -0.501339  
           ABBV    3.890018  3.937576  3.985133  0.679390  0.411808  
           ABT     3.715288  3.752833  3.790378 -0.115991  0.607718  
           ACGL    3.402675  3.430145  3.457615 -0.875012 -0.450039  
...                     ...       ...       ...       ...       ...  
2025-05-31 XYL     4.678320  4.777212  4.876105 -1.323269  1.367672  
           YUM     4.962103  4.994632  5.027161 -1.682124 -0.602270  
           ZBH     4.524274  4.595950  4.667626 -1.120584 -0.988090  
           ZBRA    5.357481  5.522473  5.687466  0.929265  0.755502  
           ZTS     4.982374  5.043721  5.105067 -0.784831  0.327079  

[47298 rows x 9 columns]

- Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [16]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2018-05-31 AAPL     44.204174         -0.001200  59.945502  3.779782   
           ABBV     72.653084         -0.041405  46.193237  4.288176   
           ABT      54.458668         -0.006269  54.184911  3.961811   
           ACN     140.315735         -0.004265  53.108546  4.925236   
           ADBE    249.279999          0.000167  67.105803  5.431916   
...                       ...               ...        ...       ...   
2025-05-31 VZ       42.650002          0.000075  47.657027  3.762462   
           WDAY    270.019989          0.000093  66.035757  5.368577   
           WFC      75.989998          0.000205  62.935699  4.151715   
           WMT      95.879997          0.000220  54.534919  4.527725   
           XOM     108.461052          0.000068  52.842848  4.633090   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2018-05-31 AAPL    3.809634  3.839485 -0.917052  0.088349  
           ABBV    4.337685  4.387195  1.791506  0.018222  
           ABT     4.004612  4.047413  0.196902  0.207945  
           ACN     4.944963  4.964690  0.226594 -0.002851  
           ADBE    5.480050  5.528185 -1.184392  0.460818  
...                     ...       ...       ...       ...  
2025-05-31 VZ      3.791511  3.820560 -1.781915 -0.010364  
           WDAY    5.491662  5.614747  0.514480  1.479676  
           WFC     4.260194  4.368673 -0.993850  1.680193  
           WMT     4.572275  4.616824 -0.575134  2.048463  
           XOM     4.669005  4.704921 -1.357163 -0.390327  

[12665 rows x 8 columns]

# 4. Calculate Monthly Returns for different time horizons as features.
- To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.

In [18]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2020-05-31 AAPL     77.264900         -0.000299  61.316184  4.275535   
           ABBV     75.506958         -0.012805  62.014871  4.217211   
           ABT      86.871468         -0.000389  57.495426  4.397978   
           ACN     187.183044         -0.001924  61.784424  5.095705   
           ADBE    386.600006          0.000447  60.939812  5.847931   
...                       ...               ...        ...       ...   
2025-05-31 VZ       42.650002          0.000075  47.657027  3.762462   
           WDAY    270.019989          0.000093  66.035757  5.368577   
           WFC      75.989998          0.000205  62.935699  4.151715   
           WMT      95.879997          0.000220  54.534919  4.527725   
           XOM     108.461052          0.000068  52.842848  4.633090   

                     bb_mid   bb_high       atr      macd  return_1m  \
date       ticker                                                      
2020-05-31 AAPL    4.334029  4.392522 -0.541878  0.681812   0.085094   
           ABBV    4.293418  4.369625  0.194277  0.756331   0.127372   
           ABT     4.440928  4.483878  0.989280  0.271575   0.030731   
           ACN     5.172519  5.249332  0.268528  1.122360   0.088720   
           ADBE    5.911794  5.975657  0.235904  1.029244   0.093202   
...                     ...       ...       ...       ...        ...   
2025-05-31 VZ      3.791511  3.820560 -1.781915 -0.010364  -0.032002   
           WDAY    5.491662  5.614747  0.514480  1.479676   0.102122   
           WFC     4.260194  4.368673 -0.993850  1.680193   0.076002   
           WMT     4.572275  4.616824 -0.575134  2.048463  -0.011704   
           XOM     4.669005  4.704921 -1.357163 -0.390327   0.036259   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2020-05-31 AAPL     0.119683   0.052595   0.030240   0.048803    0.052020  
           ABBV     0.111054   0.031444   0.013930   0.043826    0.020784  
           ABT      0.099070   0.070099   0.019104   0.013318    0.019927  
           ACN      0.108074   0.038959   0.001776   0.003329    0.011470  
           ADBE     0.102183   0.038558   0.037751   0.034656    0.030080  
...                      ...        ...        ...        ...         ...  
2025-05-31 VZ      -0.022576   0.001814  -0.000893   0.007763    0.008454  
           WDAY     0.075293   0.008385   0.012929   0.002851    0.020585  
           WFC      0.031655  -0.008209   0.001339   0.031397    0.022017  
           WMT      0.046323  -0.007610   0.007236   0.025165    0.032802  
           XOM     -0.040639  -0.005631  -0.010920  -0.006372   -0.003637  

[8431 rows x 14 columns]

# 5. Download Fama-French Factors and Calculate Rolling Factor Betas.
- We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression.

- The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models.

- We can access the historical factor returns using the pandas-datareader and estimate historical exposures using the RollingOLS rolling linear regression.

In [ ]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data